The goal is to annotate MSigDB (https://www.gsea-msigdb.org/gsea/msigdb) and SynGO (https://syngoportal.org/) gene sets to CHR:FROM-TO coordinates (in GRC37 build).
From MSigDB I'm interested in the C5 category, excluding HPO sub-category - which leaves Biological Process (BP), Cellular Component (CC) and Molecular Function (MF) ontologies, in total 10402 genesets. SynGO adds additional 73 gene sets (on top of what's already in MSigDB, and excluding small genesets of less than 5 genes). These genesets combined have 19366 unique genes.

I'm aware of four different ways of refering to a gene:
(1) gene symbol
(2) Ensembl ID
(3) Entrez ID (from NCBI)
(4) HGNC ID

C5 category in MSigGB uses Entrez ID, while SynGO uses HGNC IDs. Historically I used to get coordinates from Ensembl, because of convenient bulk export web interface ( http://grch37.ensembl.org/biomart/martview/ ).  So I've been trying to map Entrez IDs and HGNC IDs to Ensembl IDs. 

The file ``mart_export_14jan2022.txt`` in this repository is downloaded using the link above, after selecting the following columns - which include both HGNC and Entrez IDs. However the mapping is sparse and quite ambiguous.
```
Gene stable ID,Gene stable ID version,Chromosome/scaffold name,Gene description,Gene name,
Gene type,Gene Synonym,Gene start (bp),Gene end (bp),HGNC ID,NCBI gene (formerly Entrezgene) ID
```

This is easy for SynGO, as it comes with ``syngo_genes.xlsx`` containing a handy table to map HGNC to both Entrez and Ensembl. Unfortunately that table covers only 1233 genes, and I can't use it to map genes in MSigDB ongologies. I also found https://syngoportal.org/convert.html which does a great job of mapping Entrez IDs <-> HGNC IDs (also for all genes in MSigDB). I have the map in ``SynGO_id_convert/idmap.xlsx``. Unfortunately this tool doesn't map to Ensembl ID.

My current solution is to use EnsemblID -> EntrezID mapping provided by Ensembl (part of ``mart_Export_14jan2022.txt``). When EntrezID is missing, I tried to put it from ``SynGO_id_convert/idmap.xlsx``, matching based on HGNC ID. This mapped in total 19151 out of 19366 genes, leavning 215 genes unmaped. I haven't tried to use gene symbols to map the remaining genes.

I'm somewhat concerned to leave out 215 genes.

The conclusion is that EntrezID should be mapped to CHR:FROM-TO coordinates from NCBI database, without involving Ensembl. Sort of obvious, but I didn't expect that mapping humane genes across databases is going to be so difficult :)

UPD January 24

The solution turned out to be simple - it's quite easy to query NCBI databases, using esearch/efilter/efetch utilities as described here https://www.ncbi.nlm.nih.gov/books/NBK179288/ (super helpful docs!).  Some other alternatives that didn't work: https://www.ncbi.nlm.nih.gov/sites/batchentrez - helpful but can't query more than 1000 at a time, and also can't fetch hg19 coordinates. https://ftp.ncbi.nlm.nih.gov/genomes/all/annotation_releases/9606/105.20201022/ - useful resource but I couldn't figure out whether it contained gene-level information in a suitable format.

I query genes for each chromosome as follows:
```
for i in {1..22}
do
esearch -db gene -query "Homo sapiens [ORGN] AND $i [CHR]" | efilter -status alive -type coding | efetch -format docsum > esearch.chr$i.coding.results;
done
```
Then also for X, Y, and MT chromosomes:
```
esearch -db gene -query "Homo sapiens [ORGN] AND X [CHR]" | efilter -status alive -type coding | efetch -format docsum > esearch.chrX.coding.results;
esearch -db gene -query "Homo sapiens [ORGN] AND Y [CHR]" | efilter -status alive -type coding | efetch -format docsum > esearch.chrY.coding.results;
esearch -db gene -query "Homo sapiens [ORGN] AND MT [CHR]" | efilter -status alive -type coding | efetch -format docsum > esearch.chrMT.coding.results;
```


In [2]:
# read gene information from NCBI
# produces files 
# EntrezID.LocationHist.txt - info on 61837 genes (coding and pseudo)
# EntrezID.LocationHist.coding.txt - info on 19608 genes (coding only)
# the only filtering at this point is on having location information
# here is an example of gene that doesn't have known location info https://www.ncbi.nlm.nih.gov/gene/?term=401191

import xmltodict
import pandas as pd
import numpy as np

for suffix in ['', '.coding']:
    dfs=[]
    for chri in list(range(1, 23)) + ['X', 'Y', 'MT']:
        num_genes = 0
        num_genes_without_location_hist = 0

        fname=f'entrez/esearch.chr{chri}{suffix}.results'
        xml = open(fname).readlines()
        orig_lines = len(xml)
        xml = [x for x in xml if ("'" not in x)]
        diff_lines = orig_lines - len(xml)
        if diff_lines>0: print(f'remove {diff_lines} malformed lines')
        xml = xmltodict.parse('\n'.join(['<Root>'] + xml[2:] + ['</Root>']))
        DocumentSummarySet = xml['Root']['DocumentSummarySet']
        DocumentSummarySet = DocumentSummarySet if isinstance(DocumentSummarySet, list) else [DocumentSummarySet]
        for dx in DocumentSummarySet:
            for gene in dx['DocumentSummary']:
                num_genes += 1
                if 'LocationHist' not in gene: 
                    num_genes_without_location_hist += 1; continue
                x=gene['LocationHist']['LocationHistType']
                df = pd.DataFrame(x) if isinstance(x, list) else pd.DataFrame(x, index=[0])
                df['Id'] = gene['Id']
                df['Name'] = gene['Name']
                if 'GenomicInfo' in gene:
                    x = gene['GenomicInfo']['GenomicInfoType']
                    x = x if isinstance(x, list) else [x]
                    df['ExonCount'] = x[0]['ExonCount']
                else:
                    df['ExonCount'] = None
                dfs.append(df)
        print(f'chri {chri}: {num_genes} genes in total, {num_genes_without_location_hist} without location history')
    df=pd.concat(dfs)
    df['Id'] = df['Id'].astype(int)
    fname = f'EntrezID.LocationHist{suffix}.txt'
    df.to_csv(fname,sep='\t', index=False)
    print(f'save info on {len(df["Id"].unique())} genes to {fname}')

chri 1: 6368 genes in total, 230 without location history
chri 2: 4821 genes in total, 160 without location history
chri 3: 3779 genes in total, 153 without location history
chri 4: 2962 genes in total, 278 without location history
chri 5: 3252 genes in total, 343 without location history
chri 6: 3858 genes in total, 462 without location history
chri 7: 3342 genes in total, 135 without location history
chri 8: 2534 genes in total, 98 without location history
chri 9: 2660 genes in total, 92 without location history
chri 10: 2575 genes in total, 78 without location history
chri 11: 3398 genes in total, 132 without location history
chri 12: 2910 genes in total, 88 without location history
chri 13: 1586 genes in total, 85 without location history
chri 14: 2336 genes in total, 86 without location history
chri 15: 2158 genes in total, 106 without location history
chri 16: 2341 genes in total, 90 without location history
remove 2 malformed lines
chri 17: 2920 genes in total, 110 without locat

In [3]:
# Filter Enterez genes
# - grc37 annotations release, primary assembly only
# - autosomes only (not X, Y, MT)
# - with SNPs in UKB and HRC references
# Save resulting genes to EntrezID.QCed.txt

import xmltodict
import pandas as pd
import numpy as np

do_filter_by_reference = True
window_size = 10000

fname = 'EntrezID.LocationHist.txt'
print(f'reading {fname}')
ncbi_gene = pd.read_csv(fname, sep='\t', dtype=str)

fname = 'EntrezID.LocationHist.coding.txt'
print(f'reading {fname}')
ncbi_gene_coding = pd.read_csv(fname, sep='\t', dtype=str)

ncbi_gene_coding['GeneType'] = 'coding'
ncbi_gene = pd.merge(ncbi_gene, ncbi_gene_coding[['Id', 'GeneType']].drop_duplicates(), on='Id', how='left')
ncbi_gene.loc[ncbi_gene['GeneType'].isnull(), 'GeneType'] = 'pseudo'

print(f'\n{len(ncbi_gene["Id"].unique())} genes break down into coding and non-coding genes:')
print(ncbi_gene[['Id', 'GeneType']].drop_duplicates()['GeneType'].value_counts())

ncbi_gene['Id']=ncbi_gene['Id'].astype(int)
ncbi_gene['ChrStart']=ncbi_gene['ChrStart'].astype(int)
ncbi_gene['ChrStop']=ncbi_gene['ChrStop'].astype(int)

idx = (ncbi_gene['AnnotationRelease']=='105.20201022') & ncbi_gene['ChrAccVer'].str.startswith('NC_')
ncbi_gene = ncbi_gene[idx].copy()
print(f'\n{len(ncbi_gene["Id"].unique())} genes after filtering on  AnnotationRelease 105.20201022 and primary assembly\n(excluding NCBI alternate loci;  see https://www.ncbi.nlm.nih.gov/assembly/model/ for more info)')
print(ncbi_gene[['Id', 'GeneType']].drop_duplicates()['GeneType'].value_counts())

ncbi_gene = ncbi_gene[[x.split('.')[0] not in ['NC_000023', 'NC_000024', 'NC_012920'] for x in ncbi_gene['ChrAccVer'].values]].copy()
print(f'\n{len(ncbi_gene["Id"].unique())} genes after filtering onfiltering NCBI genes on chr1..22 (exclude X, Y, MT)')
print(ncbi_gene[['Id', 'GeneType']].drop_duplicates()['GeneType'].value_counts())

ncbi_gene['CHR'] = [int(x.replace('NC_', '').split('.')[0]) for x in ncbi_gene['ChrAccVer'].values]
ncbi_gene['FROM'] = np.minimum(ncbi_gene['ChrStart'].values, ncbi_gene['ChrStop'].values)
ncbi_gene['TO'] = np.maximum(ncbi_gene['ChrStart'].values, ncbi_gene['ChrStop'].values)
ncbi_gene['EZID'] =  ncbi_gene['Id']
if do_filter_by_reference:
    df = ncbi_gene.copy()
    # now filter genes that aren't present in the reference
    print(f'filter genes that are not present in the reference(s) within {window_size/1000} KB window - this step may take some time...')
    for ref_file in ['/home/oleksanf/github/mixer_private_docker/reference/hrc_EUR_qc/hrc_chr{}_EUR_qc.bim', '/home/oleksanf/github/mixer_private_docker/reference/ukb_EUR_qc/ukb_imp_chr{}_v3_qc.bim']:
        print(f'reading {ref_file}...')
        df['SNPS_in_ref'] = 0
        for chri in range(1, 23):
            #print(chri)
            ref=pd.read_csv(ref_file.format(chri), sep='\t', header=None, names='CHR SNP GP BP A1 A2'.split())
            for idx in df[df['CHR']==chri].index:
                a=df.loc[idx, 'FROM']
                b=df.loc[idx, 'TO']
                df.loc[idx, 'SNPS_in_ref'] = np.sum((ref['BP']>=a-window_size) & (ref['BP'] <=b+window_size))
        print(f'\t{np.sum(df.SNPS_in_ref!=0)} genes remain after excluding {np.sum(df.SNPS_in_ref==0)} genes which do not overlap with {ref_file}')
        df = df[df['SNPS_in_ref'] != 0].copy()
        del df['SNPS_in_ref']
    ncbi_gene = df 
print(f'\n{len(ncbi_gene["Id"].unique())} genes after on UKB and HRC references')
print(ncbi_gene[['Id', 'GeneType']].drop_duplicates()['GeneType'].value_counts())

ncbi_gene.rename(columns={'Name':'GENE'}, inplace=True)
ncbi_gene[['EZID', 'GENE', 'GeneType', 'CHR', 'FROM', 'TO']].to_csv('EntrezID.QCed.txt', sep='\t', index=False)

reading EntrezID.LocationHist.txt
reading EntrezID.LocationHist.coding.txt

61837 genes break down into coding and non-coding genes:
pseudo    42229
coding    19608
Name: GeneType, dtype: int64

47866 genes after filtering on  AnnotationRelease 105.20201022 and primary assembly
(excluding NCBI alternate loci;  see https://www.ncbi.nlm.nih.gov/assembly/model/ for more info)
pseudo    28596
coding    19270
Name: GeneType, dtype: int64

45221 genes after filtering onfiltering NCBI genes on chr1..22 (exclude X, Y, MT)
pseudo    26844
coding    18377
Name: GeneType, dtype: int64
filter genes that are not present in the reference(s) within 10.0 KB window - this step may take some time...
reading /home/oleksanf/github/mixer_private_docker/reference/hrc_EUR_qc/hrc_chr{}_EUR_qc.bim...
	44726 genes remain after excluding 495 genes which do not overlap with /home/oleksanf/github/mixer_private_docker/reference/hrc_EUR_qc/hrc_chr{}_EUR_qc.bim
reading /home/oleksanf/github/mixer_private_docker/refer

In [3]:
import xmltodict
import pandas as pd
import numpy as np

do_save = True

import os
if not os.path.exists('msigdb_v7.5.xml'):
    os.system('gunzip -c msigdb_v7.5.xml.gz >msigdb_v7.5.xml')

# read gene set definitions
# documentation: https://software.broadinstitute.org/cancer/software/gsea/wiki/index.php/MSigDB_v7.0_Release_Notes
#fname = "msigdb_v7.0.xml"
fname = "msigdb_v7.5.xml"
print(f'reading {fname}...')
xml = xmltodict.parse(open(fname).read())
cols = 'STANDARD_NAME EXACT_SOURCE MEMBERS MEMBERS_EZID CATEGORY_CODE SUB_CATEGORY_CODE'
msigdb = pd.DataFrame([tuple([x['@'+col] for col in cols.split()]) for x in xml['MSIGDB']['GENESET']], columns=cols.split())
print(f'\ttotal genesets in MSIGDB/GENESET: {len(msigdb)}')

msigdb=msigdb[msigdb['CATEGORY_CODE'].isin(['C5'])]  # ['C2', 'C5'])]
print(f'\tafter filtering on C5: {len(msigdb)}')

msigdb=msigdb[~msigdb['SUB_CATEGORY_CODE'].isin(['HPO'])] 
print(f'\tafter excluding SUB_CATEGORY_CODE HPO: {len(msigdb)}')

assert len(msigdb['STANDARD_NAME'].unique())== len(msigdb) # check that all standard names are unique
genesets = msigdb[['EXACT_SOURCE', 'STANDARD_NAME', 'MEMBERS_EZID']].copy()

fname='SynGO/syngo_ontologies.xls'
print(f'reading {fname}...')
syngo_onto=pd.read_excel(fname)
syngo_gene=pd.read_excel('SynGO/syngo_genes.xls')
syngo_anno=pd.read_excel('SynGO/syngo_annotations.xls')

# convert from hgnc gene ids to EZID
hgnc_to_enterez = dict((hgnc_id, entrez_id) for hgnc_id, entrez_id in syngo_gene[['hgnc_id', 'entrez_id']].values)
syngo_onto['MEMBERS_EZID'] = [
    ','.join([str(hgnc_to_enterez[hgnc]) for hgnc in genes_hgnc_id.split(';')])
    for genes_hgnc_id in syngo_onto['genes - hgnc_id'].values]

syngo_onto = syngo_onto[['GO term ID', 'GO term name', 'MEMBERS_EZID']]
syngo_onto.columns=['EXACT_SOURCE', 'STANDARD_NAME', 'MEMBERS_EZID']
syngo_onto['STANDARD_NAME'] = ['SYNGO_' + '_'.join(x.upper().split()) for x in syngo_onto['STANDARD_NAME'].values]
syngo_onto['GENE_COUNT'] = [len(x.split(',')) for x in syngo_onto['MEMBERS_EZID']]

print(f'\t{len(syngo_onto)} genesets in total')

syngo_onto=syngo_onto[~syngo_onto['EXACT_SOURCE'].isin(genesets['EXACT_SOURCE'].values)].copy()
print(f'\t{len(syngo_onto)} after filtering out genesets found in msigdb')

syngo_onto  = syngo_onto[syngo_onto['GENE_COUNT'] >= 5].copy()
del syngo_onto['GENE_COUNT']
print(f'\t{len(syngo_onto)} after filtering genesets with less than 5 genes (same threshold as applied in MSigDB)')

genesets = pd.concat([genesets, syngo_onto])
print(f'\t{len(genesets)} genesets in total after integrating {len(syngo_onto)} syngo genesets with msigdb')

if do_save:
    genesets.to_csv('gsa-mixer-geneset-with-exact-source_27jan2022.csv', sep='\t', index=False)

# convert to long format
go_id = 'STANDARD_NAME'
genesets = pd.DataFrame(genesets['MEMBERS_EZID'].str.split(',').tolist(), index=genesets[go_id]).stack()
genesets = genesets.reset_index([0, go_id])
genesets.columns = [go_id, 'EZID']
genesets['EZID']=genesets['EZID'].astype(int)
genesets.rename(columns={go_id:'GO'}, inplace=True)
print(f'\t{len(genesets["EZID"].unique())} unique genes across genesets')

fname = 'EntrezID.QCed.txt'
print(f'reading {fname}...')
ncbi_gene=pd.read_csv(fname, sep='\t')
print(f'\t{len(ncbi_gene)} genes in NCBI (after QC)')

ncbi_gene = ncbi_gene[ncbi_gene['GeneType']=='coding'].copy()
print(f'\t{len(ncbi_gene)} genes in NCBI (after QC, conding genes only)')

print('after merging gene and geneset definitions:')
genesets = pd.merge(genesets, ncbi_gene, on='EZID', how='inner')
print('\tgenesets:     {}'.format(len(genesets['GO'].unique())))
print('\tgenes:        {}'.format(len(genesets['GENE'].unique())))

# generate all_genes, coding_genes and pseudo_genes categories (for the baseline model)
df2=ncbi_gene.loc[ncbi_gene['GeneType']=='coding', ['GENE', 'CHR', 'FROM', 'TO']].copy()
df2['GO'] = 'coding_genes'

# put each gene as it's own gene set ('solo')
df4=ncbi_gene.copy()
df4['GO'] = df4['GENE']

# gene sets (GO + SynGO)
df5=genesets[['GO', 'GENE', 'CHR', 'FROM', 'TO']].copy()

# gene sets (GO + SynG0) leave-one-out analysis for gene sets with up to 25 genes
assert len([x for x in list(set(df5['GO'].values)) if '_excl_' in x]) == 0 # check we can use '_excl_' as a separator between geneset and 'loo' gene being excluded
df5_count = df5[['GO', 'GENE']].groupby(['GO'], as_index=False).count()
df5_count = df5_count[df5_count['GENE']<=25]
geneset_for_loo_analysis = df5_count['GO'].values
print(f'select {len(geneset_for_loo_analysis)} gene sets for leave-one-gene-out analysis; {df5_count["GENE"].mean():.2f} genes per gene set on average')
df5loo = df5[df5['GO'].isin(geneset_for_loo_analysis)].copy()
df5loo = pd.merge(df5loo[['GO', 'GENE']].rename(columns={'GENE':'excl_GENE'}), df5loo)
df5loo = df5loo[df5loo['excl_GENE'] != df5loo['GENE']]
df5loo['GO'] = [f'{go}_excl_{gene}' for go, gene in zip(df5loo['GO'].values, df5loo['excl_GENE'].values) ]
del df5loo['excl_GENE']

if do_save:
    # save the results (GSA MiXeR format)
    pd.concat([df2], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-baseline-annot_10mar2023.csv', sep='\t', index=False)
    pd.concat([df2, df4], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-gene-annot_10mar2023.csv', sep='\t', index=False)
    pd.concat([df2, df5], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-geneset-annot_10mar2023.csv', sep='\t', index=False)
    pd.concat([df2, df5, df5loo], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-genesetLOO-annot_10mar2023.csv', sep='\t', index=False)      
    pd.concat([df2, df4, df5], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-hybrid-annot_10mar2023.csv', sep='\t', index=False)
    pd.concat([df2, df4, df5, df5loo], sort=True)[['GO', 'GENE', 'CHR', 'FROM', 'TO']].to_csv('gsa-mixer-hybridLOO-annot_10mar2023.csv', sep='\t', index=False)      

    # save the resuls (MAGMA format)
    ncbi_gene['STRAND'] = '+'
    ncbi_gene[['GENE', 'CHR', 'FROM', 'TO', 'STRAND', 'GENE']].to_csv('magma-gene-annot_10mar2023.csv', sep='\t', index=False)
    genesets[['GO', 'GENE']].groupby(['GO'], as_index=False).agg({'GENE': ' '.join}).to_csv('magma-geneset-annot_10mar2023.csv', sep='\t', index=False, header=None)


reading msigdb_v7.5.xml...
	total genesets in MSIGDB/GENESET: 33365
	after filtering on C5: 15473
	after excluding SUB_CATEGORY_CODE HPO: 10402
reading SynGO/syngo_ontologies.xls...
	293 genesets in total
	184 after filtering out genesets found in msigdb
	73 after filtering genesets with less than 5 genes (same threshold as applied in MSigDB)
	10475 genesets in total after integrating 73 syngo genesets with msigdb
	19366 unique genes across genesets
reading EntrezID.QCed.txt...
	44247 genes in NCBI (after QC)
	18201 genes in NCBI (after QC, conding genes only)
after merging gene and geneset definitions:
	genesets:     10472
	genes:        16787
select 6350 gene sets for leave-one-gene-out analysis; 10.84 genes per gene set on average
